# Section 1
#   Multithreading - Difference between Process And Thread
#   Keyword - Process, Thread

(1).프로세스
    - 운영체제 -> 할당 받는 자원 단위(실행 중인 프로그램)
    - CPU동작 시간, 주소공간(독립적)
    - * Code, Data, Stack, Heap -> 독립적
    - 최소 1개의 메인스레드 보유 (프로세스 내부에 스레드가 존재)
    - 파이프, 파일, 소켓등을 사용해서 프로세스간 통신(Cost 높음)-> Context Switching

(2).스레드
    - 프로세스 내에 실행 흐름 단위
    - 프로세스 자원 사용
    - * Stack만 별도 할당 나머지는 공유(Code, Data, Heap)
    - 메모리 공유(변수 공유)
    - 한 스레드의 결과가 다른 스레드에 영향 끼침
    - 동기화 문제는 정말 주의(디버깅 어려움)

(3).멀티 스레드
    - 한 개의 단일 어플리케이션(응용프로그램) -> 여러 스레드로 구성 후 작업 처리
    - 시스템 자원 소모 감소(효율성), 처리량 증가(Cost 감소)
    - 통신 부담 감소, 디버깅 어려움, 단일 프로세스에는 효과 미약, 자원 공유 문제(교착상태), 프로세스 영향준다.

(4).멀티 프로세스
    - 한 개의 단일 어플리케이션(응용프로그램) -> 여러 프로세스로 구성 후 작업 처리
    - 한 개의 프로세스 문제 발생은 확산 없음(프로세스 Kill)
    - 캐시 체인지, Cost 비용 매우 높음(오버헤드), 복잡한 통신 방식 사용

#   Multithreading - Python's GIL
#   Keyword - CPython, 메모리관리, Gil사용 이유

#   Gil(Global Interpreter Lock)

    (1).CPython -> Python(bytecode) 실행 시 여러 thread 사용 할 경우
        단일 스레드만이 Python object에 접근 하게 제한하는 mutex
    (2).CPython 메모리 관리가 취약하기 때문(즉, Thread-safe)
    (3).단일 스레드도 충분히 빠르다.
    (4).프로세스 사용 가능(Numpy/Scipy)등 "Gil 외부 영역에서" 효율적인 코딩 가능
    (5).병렬 처리는 Multiprocessing, asyncio 선택지 다양함.
    (6).thread 동시성 완벽 처리를 위해서 -> Jython, IronPython, Stackless Python 등이 존재

In [ ]:


"""
Section 1
Multithreading - Thread(1) - Basic
Keyword - threading basic
"""

import logging
import threading
import time

# 스레드 실행 함수
def thread_func(name):
    logging.info("Sub-Thread %s: starting", name)
    time.sleep(3)
    logging.info("Sub-Thread %s: finishing", name)

# 메인 영역
# 메인 프로세스 (메인 스레드)의 시작 지점 
if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    logging.info("Main-Thread : before creating thread")
    
    # 함수 인자 확인
    x = threading.Thread(target=thread_func, args=('First',))

    logging.info("Main-Thread : before running thread")
   
    # 서브 스레드 시작
    x.start()

    logging.info("Main-Thread : wait for the thread to finish")
    
    # 주석 전후 결과 확인
    # join() : 자식 스레드 잡이 종료될 때까지 대기 
    #x.join()
    
    logging.info("Main-Thread : all done")

# Multithreading - Thread(2) - Daemon, Join
#   Keyword - DeamonThread, Join

# DaemonThread(데몬스레드)

    (1).백그라운드에서 실행
    (2).메인스레드 종료시 즉시 종료
    (3).주로 백그라운드 무한 대기 시 이벤트 발생시 실행하는 부분 담당
        - 자동저장, JVM의 Garbage Collection 등과 같은 상황에서 사용 
        - 메인스레드 종료시 같이 종료  
    (4).일반 스레드는 작업 종료시 까지 실행

In [ ]:
import logging
import threading
import time

# 스레드 실행 함수
def thread_func(name, d):
    logging.info("Sub-Thread %s: starting", name)

    #time.sleep(3)

    for i in d:
        #pass
        print(i)

    logging.info("Sub-Thread %s: finishing", name)

# 메인 영역
if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    logging.info("Main-Thread : before creating thread")
    
    # 함수 인자 확인
    # Deamon : Default False
    x = threading.Thread(target=thread_func, args=('First', range(200)), daemon=True)
    # x.daemon = True

    y = threading.Thread(target=thread_func, args=('Second', range(100)), daemon=True)
    
    logging.info("Main-Thread : before running thread")
    
    # 서브 스레드 시작
    x.start()
    y.start()

    # DaemonThread 확인
    #print(x.isDaemon())
   
    logging.info("Main-Thread : wait for the thread to finish")
    
    # 주석 전후 결과 확인
    # join() : 자식 스레드 잡이 종료될 때까지 대기 
    # x.join() 
    # y.join()
    
    logging.info("Main-Thread : all done")

# Multithreading - Thread(3) - ThreadPoolExecutor
#   Keyword - Many Threads, concurrent.futures, (xxx)PoolExcutor

# 그룹스레드
    (1).Python 3.2 이상 표준 라이브러리 사용
    (2).concurrent.futures
    (3).with사용으로 생성, 소멸 라이프사이클 관리 용이
    (4).디버깅하기가 난해함(단점)
    (5).대기중인 작업 -> Queue -> 완료 상태 조사 -> 결과 또는 예외 -> 단일화(캡슐화)

In [ ]:
import logging 
from concurrent.futures import ThreadPoolExecutor
import time

# 스레드 실행 함수
def task(name):
    logging.info("Sub-Thread %s: starting", name)

    result = 0
    for i in range(10001):
        result = result + i

    logging.info("Sub-Thread %s: finishing result: %d", name, result)

    # return 1
    # return result
    return name + ' : ' + str(result)
    # print (name + ' : ' + str(result))

# 메인 영역
def main():

    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main-Thread : before creating and running thread")

    # 실행 방법1
    # max_workers : 작업의 개수가 남어가면 직접 설정이 유리
    # executor = ThreadPoolExecutor(max_workers=3)
    
    # task1 = executor.submit(task, ('First',))
    # task2 = executor.submit(task, ('Second',))

    # 결과 값 있을 경우
    #print(task1.result())
    #print(task2.result())

    # 실행 방법2
    # with context 구문 사용

    with ThreadPoolExecutor(max_workers=2) as executor:
        tasks = executor.map(task, ['1st', '2nd', '3rd'])
        
        # 결과 확인
        print(tasks)
        print(list(tasks))

    logging.info("Main-Thread : all done")

if __name__ == '__main__':
    main()

# -------------------------------------------------------------------
# Multithreading - Thread(4) - Lock, Deadlock
#   Keyword - Lock, Deadlock, Race Condition, Thread synchronization
#   6-1,2 
# -------------------------------------------------------------------

# 용어 설명

    (1).세마포어(Semaphore) : "프로세스간 공유된 자원"에 접근 시 문제 발생 가능성 
        --> 여러개의 리소스에 여러개의 프로세스가 접근할 경우 각 리소스에 
            한 개의 프로세스만 접근 처리 고안(경쟁상태 예방) 
        --> 경쟁상태 예방 

    (2).뮤텍스(Mutex) : 공유된 자원의 데이터를 여러 스레드가 접근하는 것을 막는 것.
        --> 공유된 자원에 단일 스레드가 접근(사용) 
        --> 경쟁상태 예방

    (3).Lock : 상호 배제를 위한 잠금(Lock)처리(데이터 경쟁)

    (4).데드락(Deadlock) : 프로세스가 자원을 획득하지 못해 다음 처리를 못하는 무한 대기 상황(교착상태)

    (5).Thread synchronization(스레드 동기화)를 통해서 안정적으로 동작하게 처리한다.(동기화메소드, 동기화 블록)

    (6).Semaphore와 Mutex의 차이점?
        -> 세마포어와 뮤텍스 개체는 모두 병렬 프로그래밍 환경에서 상호 배제를 위해 사용 
        -> 뮤텍스 개체는 단일 스레드가 리소스 또는 중요 섹션을 소비 허용
        -> 세마포는 리소스에 대한 제한된 수의 동시 액세스를 허용

In [ ]:
import logging
from concurrent.futures import ThreadPoolExecutor
import time

class FakeDataStore:

    # 공유 변수(value)
    def __init__(self):
        self.value = 0 

    # 변수 업데이트 함수
    def update(self, n):
        logging.info("Thread %s: starting update", n)

        # 뮤텍스 & Lock 등 동기화(Thread synchronization) 필요 (아래 미사용 예제)
        # 스레드에서 동시에 접근해 사용하기에 원하는 결과를 얻기 어려움 
        # local_copy에 스레드들이 동시에 접근함으로써 self.value는 대부분 2 반환 
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy

        logging.info("Thread %s: finishing update", n)

if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    # 클래스 인스턴스화
    store = FakeDataStore()

    logging.info("Testing update. Starting value is %d.", store.value)

    # With Context 시작
    with ThreadPoolExecutor(max_workers=2) as executor:
        for n in ['1st', '2nd', '3rd']:
            executor.submit(store.update, n)

    logging.info("Testing update. Ending value is %d.", store.value)

In [ ]:
import logging
from concurrent.futures import ThreadPoolExecutor
import time
import threading

class FakeDataStore:
    # 공유 변수(value)
    def __init__(self):
        self.value = 0
        # Lock 선언
        self._lock = threading.Lock()

    # 변수 업데이트 함수
    def update(self, n):
        logging.info("Thread %s: starting update", n)

        # 뮤텍스 & Lock 등 동기화(Thread synchronization) 필요
        
        # ----------------
        # Lock 획득(방법1)
        # ----------------
        '''
        self._lock.acquire()
        logging.info("Thread %s has lock", n)
        
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy

        logging.info("Thread %s about to release lock", n)

        # Lock 반환
        self._lock.release()
        '''

        # ----------------
        # Lock 획득(방법2)
        # ----------------
        #'''
        with self._lock:
            logging.info("Thread %s has lock", n)

            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy

            logging.info("Thread %s about to release lock", n)
        
        logging.info("Thread %s: finishing update", n)
        #'''
        
if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    # 클래스 인스턴스화
    store = FakeDataStore()

    logging.info("Testing update. Starting value is %d.", store.value)

    # With Context 시작
    with ThreadPoolExecutor(max_workers=2) as executor:
        for n in ['1st', '2nd', '3rd', '4th']:
            executor.submit(store.update, n)

    logging.info("Testing update. Ending value is %d.", store.value)

# Multithreading - Thread(5) - Prod and Cons Using Queue
#   Keyword - 생산자 소비자 패턴(Producer/Consumer Pattern)

# Producer-Consumer Pattern
    (1).멀티스레드 디자인 패턴의 정석
    (2).서버측 프로그래밍의 핵심
    (3).주로 허리역할 중요

# Python Event 객체
    (1). Flag 초기값(0)
    (2). Set() -> 1, Clear() -> 0, Wait(1 -> 리턴, 0 -> 대기), isSet() -> 현 플래그 상태

In [1]:
import concurrent.futures
import logging
import queue
import random
import threading
import time

# -------------------------
# 생산자 (Producer)
# -------------------------
def producer(queue, event):

    """네트워크 대기 상태라 가정(서버)"""
    # 'event' 값이 '1 (Set()호출)' 이 아닐때 까지 while 문 수행 
    # 최초값 '0'       : not + '0' --> '1' ==> while 수행 
    # Set() 호출시 '1' : not + '1' --> '0' ==> while 종료
    while not event.is_set():
        message = random.randint(1, 11)
        logging.info("Producer got(Create & Sending) message: %s", message)
        queue.put(message)

    logging.info("Producer received event. Exiting")

# -------------------------
# 소비자 (Consumer)
# -------------------------
def consumer(queue, event):
    """응답 받고 소비하는 것으로 가정 or DB 저장"""
    # 'event' 값이 '1 (Set()호출)' 이 아닐때 까지 while 문 수행 
    # 최초값 '0'       : not + '0' --> '1' ==> while 수행 
    # Set() 호출시 '1' : not + '1' --> '0' ==> while 종료
    while not event.is_set() or not queue.empty():
        message = queue.get()
        logging.info( "Consumer storing(Read) message: %s (size=%d)", message, queue.qsize() )

    logging.info("Consumer received event. Exiting")

# -------------------------
# main 
# -------------------------
if __name__ == "__main__":
    # Logging format 설정
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    # 사이즈 중요
    # Producer / Consumer 스레드간의 작업량에 따라 조정 
    pipeline = queue.Queue(maxsize=10)

    # 이벤트 플래그 초기 값 0
    # event 값 '0'으로 셋팅 
    event = threading.Event()

    # With Context 시작
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline, event)
        executor.submit(consumer, pipeline, event)

        # 실행 시간 조정
        time.sleep(0.1)

        logging.info("Main: about to set event")
        
        # 프로그램 종료
        # event 값 '1'로 셋팅 
        event.set()

19:17:55: Producer got(Create & Sending) message: 10
19:17:55: Producer got(Create & Sending) message: 8
19:17:55: Consumer storing(Read) message: 10 (size=0)
19:17:55: Producer got(Create & Sending) message: 7
19:17:55: Consumer storing(Read) message: 8 (size=0)
19:17:55: Producer got(Create & Sending) message: 2
19:17:55: Consumer storing(Read) message: 7 (size=0)
19:17:55: Producer got(Create & Sending) message: 11
19:17:55: Consumer storing(Read) message: 2 (size=0)
19:17:55: Producer got(Create & Sending) message: 7
19:17:55: Consumer storing(Read) message: 11 (size=0)
19:17:55: Producer got(Create & Sending) message: 4
19:17:55: Consumer storing(Read) message: 7 (size=0)
19:17:55: Producer got(Create & Sending) message: 7
19:17:55: Consumer storing(Read) message: 4 (size=0)
19:17:55: Producer got(Create & Sending) message: 5
19:17:55: Consumer storing(Read) message: 7 (size=0)
19:17:55: Producer got(Create & Sending) message: 5
19:17:55: Consumer storing(Read) message: 5 (size=0)